
# 线型回归的简洁实现

#### 重建数据集

In [245]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])  #真实权重（ 2 个特征，对应形状为 (2,)），构造标签的线性关系
true_b = 4.2    #真实偏置
features, labels = d2l.synthetic_data(true_w, true_b, 1000)  #生成数据集，每个样本的特征和标签

#### 读取数据集

In [246]:
#定义一个帮助函数，用于把给定的数据张量打包成可迭代的“小批量”（mini-batch）数据加载器
def load_array(data_arrays, batch_size, is_train=True):  #@save   数据（二元组），每个小批量的样本数  ，是否打乱
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)  #把传入的多个张量打包成一个 PyTorch 的数据集（Dataset）实例
    return data.DataLoader(dataset, batch_size, shuffle=is_train)  #基于上面的 dataset 构造一个 DataLoader，并返回它

batch_size = 10
data_iter = load_array((features, labels), batch_size)   # 迭代器（更准确说是可迭代的 DataLoader）

###### data.DataLoader 的作用是把一个数据集（Dataset）“打包”成可迭代的迷你批量（mini-batch）数据流，并在迭代时提供常用的数据管道功能。它在训练/验证循环中充当“批量取数器”。

验证是否正常工作：

In [247]:
next(iter(data_iter))
#iter()作用：逐块读取文件、不断采集数据直到出现终止标记等。因为不依赖现成的序列，这种形式适合“流式数据”

[tensor([[-1.4552, -0.3678],
         [ 0.2187,  1.1237],
         [ 2.0692, -1.0907],
         [ 1.3897,  1.6632],
         [ 1.7918, -1.5596],
         [-1.0993,  0.3690],
         [ 1.6089, -0.0050],
         [-0.2266, -0.9119],
         [ 0.8415, -1.3622],
         [ 1.1040, -0.1516]]),
 tensor([[ 2.5376],
         [ 0.8188],
         [12.0420],
         [ 1.3110],
         [13.0784],
         [ 0.7574],
         [ 7.4524],
         [ 6.8432],
         [10.5233],
         [ 6.9255]])]

#### 定义模型

In [248]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

###### nn.Linear(2, 1)
###### - 线性层（全连接层），把输入的 2 维特征映射到 1 维输出。
###### - 数学形式：y_hat = x1 w1 + x2 w2 + b，其中 w1、w2、b 是可学习参数。

#### 初始化模型参数
###### 每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零

In [249]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

#### 定义损失函数
###### 计算均方误差使用的是MSELoss类，也称为平方范数。 默认情况下，它返回所有样本损失的平均值。

In [250]:
loss = nn.MSELoss() #

#### 定义优化算法
###### 小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [251]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

#### 训练

##### 在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:
###### 1.通过调用net(X)生成预测并计算损失l（前向传播）
###### 2.通过进行反向传播来计算梯度。
###### 3.通过调用优化器来更新模型参数。

In [252]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000217
epoch 2, loss 0.000107
epoch 3, loss 0.000109


#### 验证数据误差

In [253]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0007, -0.0002])
b的估计误差： tensor([-0.0012])
